In [40]:
class CFG:
    exps = ['026', '033']
    n_fold = 4
    debug=False
    seed = 42

# ====================================================
# Colab settings
# ====================================================
class ColabConfig:
    dataset_name = 'PPPM-stacking'
    dataset_version = 'exp' + '-'.join(CFG.exps)
    dataset_new = True  # 新しいデータセットか
    dataset_dir = None  # Kaggle Dataset にアップロードするディレクトリ
    dataset_note = '""'  # 前の版からの変更点
    in_colab = False  # colab上にデータダウンロード

In [2]:
!nvcc --version
!python -c 'import torch; print(torch.__version__) '
!python --version
print('')
!nvidia-smi
print('')
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
1.11.0+cu113
Python 3.7.13

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.


Your runtime has 13.6 gigabytes of available RAM



In [3]:
%%time
import sys
COLAB = "google.colab" in sys.modules

if COLAB:
    import os
    print('This environment is Google Colab')

    # mount drive
    from google.colab import drive
    if not os.path.isdir('/content/drive'):
        drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/kaggle/PPPM/exps')

    # kaggle api token and update kaggle api
    from google.colab import files
    if not os.path.isfile('~/.kaggle/kaggle.json'):
        # files.upload()
        !mkdir -p ~/.kaggle
        !cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
        !pip install --upgrade --force-reinstall --no-deps -q kaggle

    if ColabConfig.in_colab:
        # make directory in colab
        !mkdir -p /content/input
        !mkdir -p /content/working

        # download dataset in colab
        import zipfile, glob
        os.chdir('/content/input')
        # !kaggle competitions download -qc birdclef-2022 -p birdclef-2022
        
        # !mkdir birdclef-2022
        # !cp /content/drive/MyDrive/kaggle/BirdCLEF2022/input/birdclef-2022/* ./birdclef-2022/

        for p in glob.glob('**/*.zip', recursive=True):
            print(p)
            d, f = os.path.split(p)
            # if f in ['']:
            #     continue
            with zipfile.ZipFile(p, 'r') as zipf:
                print('unzip: ', zipf)
                zipf.extractall(d)
                print('remove: ', f)
                os.remove(p)
        
        os.chdir('/content/working')

This environment is Google Colab
Mounted at /content/drive
     |████████████████████████████████| 58 kB 2.7 MB/s 
CPU times: user 2.02 s, sys: 423 ms, total: 2.44 s
Wall time: 30.4 s


In [4]:
import os
import random
import pickle
from pathlib import Path
from psutil import cpu_count
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', None)
from tqdm.auto import tqdm
tqdm.pandas()
os.system('python -m pip install -U lightgbm')
import lightgbm as lgb
print(f"lightgbm.__version__: {lgb.__version__}")
os.system('python -m pip install optuna')
import optuna.integration.lightgbm as lgb_optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import torch
print(f"torch.__version__: {torch.__version__}")

os.system('python -m pip install sentencepiece')
os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels transformers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

lightgbm.__version__: 3.3.2
torch.__version__: 1.11.0+cu113
tokenizers.__version__: 0.12.1
transformers.__version__: 4.18.0
env: TOKENIZERS_PARALLELISM=true


In [20]:
input_dir = Path('../input/us-patent-phrase-to-phrase-matching')
output_dir = Path(f"/content/drive/MyDrive/kaggle/PPPM/output/stacking{'_'.join(CFG.exps)}")
output_dir.mkdir(exist_ok=True)

ColabConfig.dataset_dir = str(output_dir)

In [10]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


def get_logger(filename=output_dir / 'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

# LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [14]:
# 特徴量エンジニアリング
def tok_len(sentence, tokenizer):
    return len(tokenizer.tokenize(sentence))

def tokenizer_feature_engineering(df, tokenizer):
    def _tok_len(sentence):
        return tok_len(sentence, tokenizer)
    
    basecols = df.columns

    df['anchor_tok_len'] = df['anchor'].map(_tok_len)
    df['target_tok_len'] = df['target'].map(_tok_len)
    df['context_tok_len'] = df['context_text'].map(_tok_len)
    df['input_len'] = df['anchor_tok_len'] + df['target_tok_len'] + df['context_tok_len'] + 4

    df['len_anc_tgt_diff'] = df['anchor_tok_len'] - df['target_tok_len']
    df['len_anc_tgt_div'] = df['anchor_tok_len'] / df['target_tok_len']

    df['len_anc_cnt_diff'] = df['anchor_tok_len'] - df['context_tok_len']
    df['len_anc_cnt_div'] = df['anchor_tok_len'] / df['context_tok_len']

    df['len_tgt_cnt_diff'] = df['target_tok_len'] - df['context_tok_len']
    df['len_tgt_cnt_div'] = df['target_tok_len'] / df['context_tok_len']

    # 作成した特徴量とidカラムだけ返す
    usecols = [col for col in df.columns if col not in basecols]
    usecols.append('id')
    return df[usecols]

In [17]:
def make_ensemble_datasets(cfg):
    cv_scores = []
    train = pd.read_csv(input_dir / 'train.csv')

    for exp in cfg.exps:
        oof_dir = Path(f'/content/drive/MyDrive/kaggle/PPPM/output/{exp}')
        oof_df = pd.read_pickle(oof_dir / 'oof_df.pkl')
        tokenizer = AutoTokenizer.from_pretrained(oof_dir / 'tokenizer')
        
        # 特徴量と予測スコアは'pred_000'のようなカラム名にする
        feature_df = tokenizer_feature_engineering(oof_df, tokenizer)
        rename_dict = {col: col+'_'+exp for col in feature_df.columns if col != 'id'}
        feature_df = feature_df.rename(columns=rename_dict)
        
        train = train.merge(feature_df, on='id', how='left')
        train = train.merge(oof_df[['id', 'pred']].rename(columns={'pred': f'pred_{exp}'}), 
                            on='id', how='left')

        cv_scores.append(get_score(train['score'], train[f'pred_{exp}']))
    
    print(f'max score: {max(cv_scores)}')
    print(f'avg score: {sum(cv_scores) / len(cv_scores)}')
    return train

In [21]:
train = make_ensemble_datasets(CFG)
display(train.head())

max score: 0.832151085616115
avg score: 0.8266851804545338


In [26]:
# foldに分ける
# credits to: https://www.kaggle.com/code/abhishek/creating-folds-properly-hopefully-p

!pip install -q iterative-stratification
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

dfx = pd.get_dummies(train, columns=["score"]).groupby(["anchor"], as_index=False).sum()
cols = [c for c in dfx.columns if c.startswith("score_") or c == "anchor"]
dfx = dfx[cols]

mskf = MultilabelStratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=42)
labels = [c for c in dfx.columns if c != "anchor"]
dfx_labels = dfx[labels]
dfx["fold"] = -1

for fold, (trn_, val_) in enumerate(mskf.split(dfx, dfx_labels)):
    print(len(trn_), len(val_))
    dfx.loc[val_, "fold"] = fold

train = train.merge(dfx[["anchor", "fold"]], on="anchor", how="left")
print(train.fold.value_counts())

550 183
549 184
550 183
550 183
3    9622
0    9379
1    8860
2    8612
Name: fold, dtype: int64


In [27]:
# カテゴリ変数の変換
train['section'] = train['context'].map(lambda s: s[0])
cat_cols = ['section']
for c in cat_cols:
    le = LabelEncoder()
    le.fit(train[c])
    train[f'{c}_encoded'] = le.transform(train[c])
    with open(output_dir / f'{c}_encoder.pkl', 'wb') as f:
        pickle.dump(le, f)

In [28]:
print(train.columns)

Index(['id', 'anchor', 'target', 'context', 'score', 'anchor_tok_len_026', 'target_tok_len_026', 'context_tok_len_026', 'input_len_026', 'len_anc_tgt_diff_026', 'len_anc_tgt_div_026', 'len_anc_cnt_diff_026', 'len_anc_cnt_div_026', 'len_tgt_cnt_diff_026', 'len_tgt_cnt_div_026', 'pred_026', 'anchor_tok_len_033', 'target_tok_len_033', 'context_tok_len_033', 'input_len_033', 'len_anc_tgt_diff_033', 'len_anc_tgt_div_033', 'len_anc_cnt_diff_033', 'len_anc_cnt_div_033', 'len_tgt_cnt_diff_033', 'len_tgt_cnt_div_033', 'pred_033', 'fold', 'section', 'section_encoded'], dtype='object')


In [29]:
excluded_columns = ['id', 'anchor', 'target', 'context', 'score', 'fold', 'section']
cols = [col for col in train.columns if col not in excluded_columns]
with open(output_dir / 'cols.pkl', 'wb') as f:
    pickle.dump(cols, f)
print(cols)

['anchor_tok_len_026', 'target_tok_len_026', 'context_tok_len_026', 'input_len_026', 'len_anc_tgt_diff_026', 'len_anc_tgt_div_026', 'len_anc_cnt_diff_026', 'len_anc_cnt_div_026', 'len_tgt_cnt_diff_026', 'len_tgt_cnt_div_026', 'pred_026', 'anchor_tok_len_033', 'target_tok_len_033', 'context_tok_len_033', 'input_len_033', 'len_anc_tgt_diff_033', 'len_anc_tgt_div_033', 'len_anc_cnt_diff_033', 'len_anc_cnt_div_033', 'len_tgt_cnt_diff_033', 'len_tgt_cnt_div_033', 'pred_033', 'section_encoded']


In [30]:
params = {
    'objective': 'regression',
    'boosting': 'gbdt',
    'num_iterations': 1000,
    'learning_rate': 0.01,
    # 'num_leaves': 20,  # 31
    'num_threads': cpu_count(logical=False),
    'seed': CFG.seed,

    # 'max_depth': 3,  # -1
    # 'min_data_in_leaf': 100,  # 20
    'verbosity': -1,  # 1

    'metric': 'l2'
}

In [31]:
def tuning_params(df, params):
    anchors = df['anchor'].unique()
    rng = np.random.default_rng(seed=71)
    rng.shuffle(anchors)
    val_prop = 0.25  # バリデーションデータの割合。4foldから0.25
    val_size = int(len(anchors)*val_prop)
    val_anchors = anchors[:val_size]
    is_val = np.isin(df['anchor'], val_anchors)
    idx = np.arange(len(df))
    val_idx = idx[is_val]
    tr_idx = idx[~is_val]
    print(f'train: {len(tr_idx)}, val: {len(val_idx)}')

    tr_df = df.iloc[tr_idx]
    val_df = df.iloc[val_idx]

    cat_cols_ = [f'{c}_encoded' for c in cat_cols]
    tr_ds = lgb.Dataset(tr_df[cols], tr_df['score'], categorical_feature=cat_cols_)
    val_ds = lgb.Dataset(val_df[cols], val_df['score'], categorical_feature=cat_cols_, reference=tr_ds)
    
    model = lgb_optuna.train(
        params,
        tr_ds, 
        valid_sets=[tr_ds, val_ds],
        callbacks=[lgb.early_stopping(stopping_rounds=100),
                    lgb.log_evaluation(period=100)],
        verbosity=-1,
    )

    best_params = model.params

    return best_params

In [32]:
best_params = tuning_params(train, params)
display(best_params)

Output hidden; open in https://colab.research.google.com to view.

In [33]:
def prepare_datasets(df, fold):
    tr_df = df[df['fold']!=fold]
    val_df = df[df['fold']==fold]

    cat_cols_ = [f'{c}_encoded' for c in cat_cols]
    tr_ds = lgb.Dataset(tr_df[cols], tr_df['score'], categorical_feature=cat_cols_)
    val_ds = lgb.Dataset(val_df[cols], val_df['score'], categorical_feature=cat_cols_, reference=tr_ds)

    return tr_ds, val_ds


def train_fn(df, params):
    preds = []

    for fold in df['fold'].unique():
        tr_ds, val_ds = prepare_datasets(df, fold)
        
        model = lgb.train(
            params, 
            tr_ds, 
            valid_sets=[tr_ds, val_ds],
            callbacks=[lgb.early_stopping(stopping_rounds=100),
                       lgb.log_evaluation(period=100)]
        )

        pred = df[df['fold']==fold].copy()
        pred['pred_lgb'] = model.predict(pred[cols], num_iteration=model.best_iteration)
        preds.append(pred)

        model.save_model(output_dir / f'lgb_fold{fold}.txt', num_iteration=model.best_iteration)

    return pd.concat(preds, axis=0, ignore_index=False)

In [34]:
pred = train_fn(train, best_params)

Training until validation scores don't improve for 100 rounds
[100]	training's l2: 0.0268073	valid_1's l2: 0.0257177
[200]	training's l2: 0.0204029	valid_1's l2: 0.0192488
[300]	training's l2: 0.0193275	valid_1's l2: 0.018222
[400]	training's l2: 0.0190625	valid_1's l2: 0.0180319
[500]	training's l2: 0.0189506	valid_1's l2: 0.018028
Early stopping, best iteration is:
[446]	training's l2: 0.0190038	valid_1's l2: 0.0180185
Training until validation scores don't improve for 100 rounds
[100]	training's l2: 0.0263487	valid_1's l2: 0.0273769
[200]	training's l2: 0.0198654	valid_1's l2: 0.0209343
[300]	training's l2: 0.0187902	valid_1's l2: 0.0198547
[400]	training's l2: 0.0185442	valid_1's l2: 0.0196565
[500]	training's l2: 0.0184351	valid_1's l2: 0.0196185
[600]	training's l2: 0.0183631	valid_1's l2: 0.019634
Early stopping, best iteration is:
[536]	training's l2: 0.0184082	valid_1's l2: 0.0196101
Training until validation scores don't improve for 100 rounds
[100]	training's l2: 0.026083	va

In [35]:
importances = []
for fold in train['fold'].unique():
    importances.append(
        lgb.Booster(model_file=output_dir / f'lgb_fold{fold}.txt').feature_importance(importance_type='gain'))
importance_df = pd.DataFrame({
    'feature': cols,
    'importance': np.mean(importances, axis=0)
    })
importance_df.sort_values('importance', ascending=False)

,feature,importance
10,pred_026,17706.894500
21,pred_033,9465.894934
22,section_encoded,48.210624
1,target_tok_len_026,38.528147
16,len_anc_tgt_div_033,24.185815
15,len_anc_tgt_diff_033,17.785597
0,anchor_tok_len_026,17.198718
11,anchor_tok_len_033,15.619125
9,len_tgt_cnt_div_026,13.417823
5,len_anc_tgt_div_026,10.386112


In [36]:
# optunaのbest params
print(get_score(pred['score'], pred['pred_lgb']))
pred.head()

0.8450698931469711


,id,anchor,target,context,score,anchor_tok_len_026,target_tok_len_026,context_tok_len_026,input_len_026,len_anc_tgt_diff_026,len_anc_tgt_div_026,len_anc_cnt_diff_026,len_anc_cnt_div_026,len_tgt_cnt_diff_026,len_tgt_cnt_div_026,pred_026,anchor_tok_len_033,target_tok_len_033,context_tok_len_033,input_len_033,len_anc_tgt_diff_033,len_anc_tgt_div_033,len_anc_cnt_diff_033,len_anc_cnt_div_033,len_tgt_cnt_diff_033,len_tgt_cnt_div_033,pred_033,fold,section,section_encoded,pred_lgb
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15,0.440476,1,3,22,30,-2,0.333333,-21,0.045455,-19,0.136364,0.413867,0,A,0,0.429089
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25,0.641294,1,3,22,30,-2,0.333333,-21,0.045455,-19,0.136364,0.531482,0,A,0,0.588181
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.247038,1,2,22,29,-1,0.500000,-21,0.045455,-20,0.090909,0.237905,0,A,0,0.246035
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.494176,1,2,22,29,-1,0.500000,-21,0.045455,-20,0.090909,0.561054,0,A,0,0.503445
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10,0.011766,1,2,22,29,-1,0.500000,-21,0.045455,-20,0.090909,0.028578,0,A,0,0.084716


In [42]:
import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi

ID = 'hanejiyuto'
DATASET_ID = ColabConfig.dataset_name + '-' + ColabConfig.dataset_version
UPLOAD_DIR = ColabConfig.dataset_dir
VERSION_NOTES = ColabConfig.dataset_note

def dataset_create_new():
    dataset_metadata = {}
    dataset_metadata['id'] = f'{ID}/{DATASET_ID}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = DATASET_ID
    with open(os.path.join(UPLOAD_DIR, 'dataset-metadata.json'), 'w') as f:
        json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    # api.dataset_create_new(folder=UPLOAD_DIR, convert_to_csv=False, dir_mode='tar')
    !kaggle datasets create -t -p $UPLOAD_DIR -r tar

def dataset_create_version():  # バージョンアップデート
    dataset_metadata = {}
    dataset_metadata['id'] = f'{ID}/{DATASET_ID}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = DATASET_ID
    if not os.path.exists('dataset-metadata.json'):
        with open(os.path.join(UPLOAD_DIR, 'dataset-metadata.json'), 'w') as f:
            json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    # api.dataset_create_version(folder=UPLOAD_DIR, version_notes=VERSION_NOTES, convert_to_csv=False, dir_mode='tar')
    !kaggle datasets version -t -p $UPLOAD_DIR -r tar -m $VERSION_NOTES

if CFG.debug:
    pass
elif ColabConfig.dataset_new:
    dataset_create_new()
else:
    dataset_create_version()

Starting upload for file section_encoder.pkl
100% 323/323 [00:00<00:00, 1.42kB/s]
Upload successful: section_encoder.pkl (323B)
Starting upload for file cols.pkl
100% 570/570 [00:00<00:00, 1.97kB/s]
Upload successful: cols.pkl (570B)
Starting upload for file lgb_fold0.txt
100% 351k/351k [00:00<00:00, 1.66MB/s]
Upload successful: lgb_fold0.txt (351KB)
Starting upload for file lgb_fold1.txt
100% 421k/421k [00:00<00:00, 1.70MB/s]
Upload successful: lgb_fold1.txt (421KB)
Starting upload for file lgb_fold3.txt
100% 343k/343k [00:00<00:00, 1.36MB/s]
Upload successful: lgb_fold3.txt (343KB)
Starting upload for file lgb_fold2.txt
100% 367k/367k [00:00<00:00, 1.49MB/s]
Upload successful: lgb_fold2.txt (367KB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/hanejiyuto/PPPM-stacking-exp026-033
